# Import necessary libraries

In [132]:
import pandas as pd

# Load data

In [133]:
data = pd.read_csv('../raw/data_raw.csv', encoding='ANSI')
item_master_data = pd.read_csv('../raw/item_master.csv', encoding='ANSI')

# Drop unnecessary columns

In [134]:
if(len(data.columns) == 17):
    data.drop(['Branch', 'Cost Rate', 'Schm.Disc ', 'Bill Disc ', 'Doctor Name ', 'Patient Name ', 'Scheduler ', 'Batch No '], axis=1, inplace=True)

# Rename necessary common column

In [135]:
data.rename(columns={'Prod Code': 'Product Code', ' ': 'Customer ID'}, inplace=True)

# Merge Dataframes to get subcat name

In [136]:
merged_df = pd.merge(data, item_master_data[['Product Code', 'SUB CAT NAME']], on='Product Code', how='inner')
merged_df

,Date,Bill No,Product Code,Prod Name,Qty,Sale Rate,MRP,Amount,Customer ID,SUB CAT NAME
0,01-Mar-18,136200,7402,AMUL FRESH MILK GOLD PP 500ML,8.0,26.0,26.0,208.0,NaN,DAIRY
1,01-Mar-18,136201,7402,AMUL FRESH MILK GOLD PP 500ML,2.0,26.0,26.0,52.0,NaN,DAIRY
2,01-Mar-18,136205,7402,AMUL FRESH MILK GOLD PP 500ML,2.0,26.0,26.0,52.0,NaN,DAIRY
3,01-Mar-18,136210,7402,AMUL FRESH MILK GOLD PP 500ML,2.0,26.0,26.0,52.0,1996.0,DAIRY
4,01-Mar-18,136219,7402,AMUL FRESH MILK GOLD PP 500ML,2.0,26.0,26.0,52.0,NaN,DAIRY
...,...,...,...,...,...,...,...,...,...,...
894629,29-Apr-17,11279,12927,BARBIE TLC SPRING ROSE 200GM,1.0,100.0,100.0,100.0,975.0,DEO & ANTIPERSPIRANTS
894630,29-Apr-17,11556,16712,AL FAHAD MTN MEAT BALL 250GM,1.0,250.0,250.0,250.0,NaN,COLD & FROZEN FOODS
894631,30-Apr-17,11829,7420,NILONS CUT SEWAIYAN 175GM,1.0,15.0,15.0,15.0,NaN,INSTANT MEALS
894632,30-Apr-17,11954,7592,CANTINA 8 CORN TORTILLAS 200G,1.0,295.0,295.0,295.0,1555.0,GOURMET ETHNIC CUISINES


# Remove NA customers and 1 Customers

In [137]:
merged_df.dropna(subset=['Customer ID'], inplace=True)
mergded_df = merged_df[merged_df['Customer ID'] != 1]
merged_df.to_csv('new_transformed_data.csv')